#### Setup 
We first set the notebook to display the output from each code block, <br>
then import the required packages and set the file directory

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from mpl_toolkits.basemap import Basemap
from shapely.geometry import Point
import matplotlib.pyplot as plt
import pandas as pd
import geopandas
import pyproj

file_dir=r'C:/Users/Mark.Rademaker/PycharmProjects/InternshipNaturalis/trait-geo-diverse-dl/pilot'

#### Read in species data
A text file with the names of all the species we want to create the occurrence maps for is loaded in. <br>
This can be used to call up the set of filtered occurrences for each species, and store this information in a dictionary.<br>


In [ ]:
#access file with list of taxa names
taxa=pd.read_csv(file_dir+"/data/spec_filtered/taxa.txt",header=None)
taxa.columns=["taxon"]
species_occ_dict={}


#store species occurrences in dictionary
for i in taxa["taxon"]:
    taxon_data = pd.read_csv(file_dir+"/data/spec_filtered/%s.csv"%i)
    species_occ_dict["%s"%i] = taxon_data  
    
#check whether all species have been included and inspect dictionary
if len(species_occ_dict.keys())==len(taxa["taxon"]):
    print("All species dataframes now in dictionary")
else:
    print("Error: not all species dataframe included")

#### Plot species occurrence points
For each species, the occurrence data is extracted from the dictionary. <br>
The centre-point of the map is determined based on the mean latitude and longitude of the occurrences. <br>
After which the basemap is created and the points are plotted on this map.

In [ ]:
%%capture
for key in species_occ_dict:
    
    #extract occurrence data for species
    data = species_occ_dict[key]
    spec = data["taxon_name"][0]
    spec=spec.replace(" ","_")
    
    lat=data["decimal_latitude"]
    lon=data["decimal_longitude"]
    
    meanlat=sum(lat)/len(lat)
    meanlon=sum(lon)/len(lon)
       
   
    #create the map extend
    fig=plt.figure(figsize=(12, 12))
    map = Basemap(projection="aeqd",resolution="l", ellps="WGS84",
        lat_0=meanlat,lon_0=meanlon,
        width=3000*2500,height=3000*2500,
        suppress_ticks=False,)  
    map.drawmapboundary(fill_color='white')
    map.fillcontinents(color='lightgray',lake_color='white')
    map.drawcountries(color="gray")
    map.drawcoastlines(linewidth=0.2)

    #plot the points on the map
    for i in range(0,len(lon)):
        xpt,ypt = map(lon[i],lat[i])
        map.plot([xpt],[ypt],"ko",markersize=5,color="yellow",markeredgecolor="black")
        
        
    #save the figure
    spec_t=spec.replace("_"," ")
    plt.title("Filtered occurrences for %s"%spec_t,fontsize=20,fontweight="bold")
    plt.savefig(file_dir+'/results/{}/{}_occurrence_map.png'.format(spec,spec),pad_inches=0.0, bbox_inches='tight')
    
    #check figure and close it
    plt.show()
    plt.close()